### TOBIG'S 14기 정규세션 4주차 SVM 
### ASSIGNMENT1. Multiclass SVM 구현

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

#IRIS 데이터 로드
iris =  sns.load_dataset('iris') 
X= iris.iloc[:,:4] #학습할데이터
y = iris.iloc[:,-1] #타겟
print(y)

0         setosa
1         setosa
2         setosa
3         setosa
4         setosa
         ...    
145    virginica
146    virginica
147    virginica
148    virginica
149    virginica
Name: species, Length: 150, dtype: object


In [2]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=48)

In [3]:
def standardization(train, test):
    scaler = StandardScaler()
    train = scaler.fit_transform(train)
    test = scaler.transform(test)
    return train, test

X_train, X_test = standardization(X_train, X_test)

### 코드의 재사용성을 위해 label의 class수와 무관한 분류기를 OVR형식으로 구현

In [4]:
class SVM_OVR:
    def __init__(self, num_classes, kernel, C, gamma):
        self.num_classes = num_classes
        self.clfs = [SVC(kernel = kernel, C = C, gamma = gamma) for _ in range(num_classes)]
        self.classes = None
        
    def fit(self, X_train, y_train):
        y_train = pd.get_dummies(y_train)
        for i in range(self.num_classes):
            self.clfs[i].fit(X_train,y_train.iloc[:,i]) 
            # 각 클래스별로 인지 아닌지를 판단하는 분류기를 학습시킵니다.
        self.classes = y_train.columns
    
    def predict(self, X_test):
        pred_df = pd.DataFrame([svm.predict(X_test) for svm in self.clfs]).T # 각 클래스 별 예측값
        decisions = np.array([svm.decision_function(X_test) for svm in self.clfs]).T # 각 클래스 별 거리
        
        final_pred = []
        for i in range(len(pred_df)):
            # 예측 중 하나의 클래스만 맞다고 판단한 경우
            # 맞다고 판단도니 클래스를 final_pred 리스트에 넣어준다
            if sum(pred_df.iloc[i]) == 1:
                label = pred_df.iloc[i][pred_df.iloc[i] == 1].index[0]
                final_pred.append(self.classes[label])
            
            # 두개 이상 혹은 전부 아니라고 판단한 경우
            # 결정경계를 이용한다.
            
            # case1 : 예측 중 두개 이상의 클래스가 맞다고 판단한 경우
            #         맞다고 판단된 클래스 중 결정경계로부터 더 먼 클래스를 리스트에 넣어준다.
            #         맞는 클래스의 경우 결정경계로부터 거리가 양수로 나타나므로 argmax를 이용
            
            # case2 : 예측 중 전부 아니라고 판단한 경우
            #         결정경계로부터 가장 가까운 클래스를 리스트에 넣어준다.
            #         전부 아니라고 판단햇으므로 결정경계로부터 거리가 모두 음수이므로 argmax이용
            
            # 두가지 케이스를 살펴본 결과
            # 모두 결정경계로부터 거리의 argmax를 final label로 채택할 수 있음
            
            else:
                label = np.argmax(decisions[i])
                final_pred.append(self.classes[label])
        
        return final_pred

In [5]:
clf = SVM_OVR(num_classes = 3, kernel = 'rbf', C = 5, gamma = 5)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [6]:
for gt, pr in zip(y_test, pred):
    print('%s%20s'%(gt, pr))

versicolor          versicolor
versicolor          versicolor
virginica          versicolor
setosa           virginica
versicolor           virginica
virginica           virginica
setosa              setosa
virginica           virginica
setosa              setosa
versicolor          versicolor
virginica           virginica
setosa              setosa
setosa              setosa
virginica           virginica
versicolor          versicolor
versicolor          versicolor
setosa              setosa
versicolor          versicolor
versicolor           virginica
virginica           virginica
setosa              setosa
virginica           virginica
versicolor          versicolor
versicolor          versicolor
virginica           virginica
setosa              setosa
setosa              setosa
virginica           virginica
virginica           virginica
versicolor          versicolor


In [7]:
accuracy_score(y_test, pred)

0.8666666666666667

### 코드의 재사용성을 위해 label의 class수와 무관한 분류기를 OVO형식으로 구현

In [8]:
from itertools import combinations

In [9]:
class SVM_OVO:
    def __init__(self, num_classes, kernel, C, gamma):
        self.num_classes = num_classes
        self.clfs = [{'class' : None,'clf' : SVC(kernel = kernel, C = C, gamma = gamma)} for _ in range(int(num_classes * (num_classes-1) / 2))]
        # num_classes의 조합의 수만큼 분류기를 만듭니다.
        self.classes = None
        self.combi = []
        
    def fit(self, X_train, y_train):
        self.classes = y_train.unique()
        i = 0
        # classes의 조합 별로 idx를 나눠 각 조합별로 svm을 훈련시킵니다.
        for c in combinations(self.classes, 2):
            idx = (y_train == c[0]) | (y_train == c[1])
            self.clfs[i]['clf'].fit(X_train[idx], y_train[idx])
            self.clfs[i]['class'] = c
            self.combi.append(c)
            i += 1
            
    def predict(self, X_test):
        preds_df = pd.DataFrame([svm['clf'].predict(X_test) for svm in self.clfs]).T # 각 조합 별 예측
        decisions = pd.DataFrame([svm['clf'].decision_function(X_test) for svm in self.clfs]).T # 각 클래스 별 거리
        decisions.columns = self.combi
        
        final_pred = []
        for i in range(len(preds_df)):
            
            # 예측들 중 가장 많은 한 클래스가 있을 경우
            # 해당 클래스를 리스트에 넣어준다
            if preds_df.iloc[i].value_counts().iloc[0] > preds_df.iloc[i].value_counts().iloc[1]:
                label = (preds_df.iloc[i].value_counts() / len(preds_df.iloc[i])).index[0]
                final_pred.append(label)
            
            # 겹치는 클래스가 존재하거나 모든 클래스가 다를 경우
            # 클래스에 해당하는 결정경계로부터 거리의 합이 큰 클래스를 선택한다
            else:
                decision_for_row = {key : 0 for key in classes}
                for c, d in zip(decisions.iloc[i].index, decisions.iloc[i]):
                    if d > 0:
                        decision_for_row[c[0]] += d
                    else:
                        decision_for_row[c[1]] -= d
                label = sorted(decision_for_row.items(), key = lambda x : x[1], reverse = True)[0][0]
                final_pred.append(label)
        return final_pred


In [10]:
clf = SVM_OVO(num_classes = 3, kernel = 'rbf', C = 5, gamma = 5)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [11]:
for gt, pr in zip(y_test, pred):
    print('%s%20s'%(gt, pr))

versicolor          versicolor
versicolor          versicolor
virginica          versicolor
setosa           virginica
versicolor           virginica
virginica           virginica
setosa              setosa
virginica           virginica
setosa              setosa
versicolor          versicolor
virginica           virginica
setosa              setosa
setosa              setosa
virginica           virginica
versicolor          versicolor
versicolor          versicolor
setosa              setosa
versicolor          versicolor
versicolor           virginica
virginica           virginica
setosa              setosa
virginica           virginica
versicolor          versicolor
versicolor          versicolor
virginica           virginica
setosa              setosa
setosa              setosa
virginica           virginica
virginica           virginica
versicolor          versicolor


In [12]:
accuracy_score(y_test, pred)

0.8666666666666667

### sklearn 라이브러리를 활용한 multi class svm

In [13]:
# 원래 라이브러리가 제공하는 multi class SVM과 여러분이 구현한 multiclass SVM 결과를 비교해주세요
svm = SVC(kernel ='rbf', C = 5, gamma = 5)
svm.fit(X_train, y_train)
y_pred = svm.predict(X_test)

accuracy_score(y_test,y_pred)

0.8666666666666667

#### 구현한 SVM_OVR, SVM_OVO를 다른 데이터셋에 적용

In [14]:
from sklearn.datasets import load_digits

In [15]:
mnist = load_digits()
X = pd.DataFrame(mnist.data)
y = pd.Series(mnist.target)

In [16]:
#간단하게 16으로 나누어 데이터를 정규화합니다.
X /= 16

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=317)

#### OVR

In [18]:
clf = SVM_OVR(num_classes = 10, kernel = 'rbf', C = 1, gamma = 1)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [19]:
accuracy_score(y_test, pred)

0.9888888888888889

#### OVO

In [20]:
clf = SVM_OVO(num_classes = 10, kernel = 'rbf', C = 1, gamma = 1)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [21]:
accuracy_score(y_test, pred)

0.9805555555555555

- 다른 데이터 셋에도 잘 작동하는것을 확인함
- OVO의 경우 조합의 수만큼 분류기를 만들기 때문에 OVR에 비해 시간이 더 많이 걸림
- 보통 OVO의 성능이 더 좋다고 알려져있지만 OVO의 성능이 OVR보다 떨어짐. 단순히 거리의 합을 기준으로 하기 때문일것이라 판단됨.
- ovo 개선점 : svm의 predict_proba기능을 사용하여 확률의 합을 이용해 볼 수 있을것같다.